In [1]:
import pandas
from IPython.display import display
from tqdm.notebook import tqdm

import automol
import chemkin_io
import helpers
import ioformat

tqdm.pandas()

In [2]:
# 1a. read in the species lists
ORIG_C67_SPC_PATH = 'data/mechanisms/05_orig_nuig-c6-7.csv'
ORIG_PYR_SPC_PATH = 'data/mechanisms/05_orig_nuig-pyro.csv'
FUEX_C67_SPC_PATH = 'data/mechanisms/05_fu-exp_nuig-c6-7.csv'
FUEX_PYR_SPC_PATH = 'data/mechanisms/05_fu-exp_nuig-pyro.csv'
NREX_C67_SPC_PATH = 'data/mechanisms/06_nr-exp_nuig-c6-7.csv'
NREX_PYR_SPC_PATH = 'data/mechanisms/06_nr-exp_nuig-pyro.csv'

ORIG_C67_SPC_DF = pandas.read_csv(ORIG_C67_SPC_PATH, quotechar="'")
ORIG_PYR_SPC_DF = pandas.read_csv(ORIG_PYR_SPC_PATH, quotechar="'")
FUEX_C67_SPC_DF = pandas.read_csv(FUEX_C67_SPC_PATH, quotechar="'")
FUEX_PYR_SPC_DF = pandas.read_csv(FUEX_PYR_SPC_PATH, quotechar="'")
NREX_C67_SPC_DF = pandas.read_csv(NREX_C67_SPC_PATH, quotechar="'")
NREX_PYR_SPC_DF = pandas.read_csv(NREX_PYR_SPC_PATH, quotechar="'")

ORIG_C67_SPC_DF['orig-name'] = ORIG_C67_SPC_DF['name']
ORIG_PYR_SPC_DF['orig-name'] = ORIG_PYR_SPC_DF['name']

In [3]:
# 1b. read in the mechanism reaction lists
ORIG_C67_RXN_PATH = 'data/mechanisms/05_orig_nuig-c6-7.txt'
ORIG_PYR_RXN_PATH = 'data/mechanisms/05_orig_nuig-pyro.txt'
C67_RXN_PATH = 'data/mechanisms/05_fu-exp_nuig-c6-7.txt'
PYR_RXN_PATH = 'data/mechanisms/05_fu-exp_nuig-pyro.txt'
NREX_C67_RXN_PATH = 'data/mechanisms/06_nr-exp_nuig-c6-7.txt'
NREX_PYR_RXN_PATH = 'data/mechanisms/06_nr-exp_nuig-pyro.txt'

ORIG_C67_MECH_STR = ioformat.pathtools.read_file('.', ORIG_C67_RXN_PATH)
ORIG_PYR_MECH_STR = ioformat.pathtools.read_file('.', ORIG_PYR_RXN_PATH)
FUEX_C67_MECH_STR = ioformat.pathtools.read_file('.', C67_RXN_PATH)
FUEX_PYR_MECH_STR = ioformat.pathtools.read_file('.', PYR_RXN_PATH)
NREX_C67_MECH_STR = ioformat.pathtools.read_file('.', NREX_C67_RXN_PATH)
NREX_PYR_MECH_STR = ioformat.pathtools.read_file('.', NREX_PYR_RXN_PATH)

ORIG_C67_RXN_DCT = chemkin_io.parser.mechanism.reactions(ORIG_C67_MECH_STR)
ORIG_PYR_RXN_DCT = chemkin_io.parser.mechanism.reactions(ORIG_PYR_MECH_STR)
FUEX_C67_RXN_DCT = chemkin_io.parser.mechanism.reactions(FUEX_C67_MECH_STR)
FUEX_PYR_RXN_DCT = chemkin_io.parser.mechanism.reactions(FUEX_PYR_MECH_STR)
NREX_C67_RXN_DCT = chemkin_io.parser.mechanism.reactions(NREX_C67_MECH_STR)
NREX_PYR_RXN_DCT = chemkin_io.parser.mechanism.reactions(NREX_PYR_MECH_STR)

In [4]:
# 1c. read in full reaction lists with reaction objects (for identifying fleeting stereo)
C67_RXN_PATH = 'data/reactions/04_reactions-fu-exp_nuig-c6-7.csv'
PYR_RXN_PATH = 'data/reactions/04_reactions-fu-exp_nuig-pyro.csv'

C67_RXN_DF = pandas.read_csv(C67_RXN_PATH, quotechar="'")
PYR_RXN_DF = pandas.read_csv(PYR_RXN_PATH, quotechar="'")

In [5]:
# 2a. function: add stats to fully expanded species DataFrame
def species_stats_columns(spc_df):
    spc_df['graph'] = spc_df['inchi'].progress_apply(automol.chi.graph)
    spc_df['pure_inchi'] = spc_df['graph'].progress_apply(automol.graph.inchi)
    spc_df['any_stereo'] = spc_df['graph'].progress_apply(automol.graph.has_stereo)
    spc_df['atom_stereo'] = spc_df['graph'].progress_apply(automol.graph.has_atom_stereo)
    spc_df['bond_stereo'] = spc_df['graph'].progress_apply(automol.graph.has_bond_stereo)
    spc_df['res_bond_stereo'] = spc_df['graph'].progress_apply(automol.graph.has_resonance_bond_stereo)
    spc_df['vin_bond_stereo'] = spc_df['graph'].progress_apply(automol.graph.has_vinyl_bond_stereo)
    spc_df['non_inchi_stereo'] = spc_df['graph'].progress_apply(automol.graph.has_noninchi_stereo)
    return spc_df

In [6]:
# 2b. function: identify reactions with fleeting stereo from the fully-expanded list
def fleeting_stereo_stats_columns(rxn_df):
    def _has_fleeting_stereo(df_):
        row = df_.iloc[0]
        rxn_obj_str = ioformat.pathtools.read_file(".", row["path"])
        rxn_obj = automol.reac.from_string(rxn_obj_str)
        ts_gra = automol.reac.ts_graph(rxn_obj)
        atm_ste, bnd_ste = automol.graph.has_fleeting_atom_or_bond_stereo(ts_gra)
        row["fleeting_atom_stereo"] = atm_ste
        row["fleeting_bond_stereo"] = bnd_ste
        return row

    cols = [
        "name",
        "ts_amchi",
        "orig-name",
        "fleeting_atom_stereo",
        "fleeting_bond_stereo",
    ]
    fle_df = rxn_df.groupby(["name", "ts_amchi"]).progress_apply(_has_fleeting_stereo)[
        cols
    ]
    fle_df.reset_index(drop=True, inplace=True)
    return fle_df

In [7]:
# 2c. function: form DataFrames with reaction stats
def reaction_stats_columns(rxn_dct, spc_df):
    def original_reaction_name(rxn_key):
        rct_names, prd_names, third_body = rxn_key
        rct_names0 = helpers.lookup_value_series(
            spc_df, "orig-name", [{"name": n} for n in rct_names]
        )
        prd_names0 = helpers.lookup_value_series(
            spc_df, "orig-name", [{"name": n} for n in prd_names]
        )
        rxn_name0 = chemkin_io.writer.format_rxn_name(
            (rct_names0, prd_names0, third_body)
        )
        return rxn_name0

    def any_reagent_(col):
        def any_reagent(names):
            return any(
                helpers.lookup_value_series(spc_df, col, [{"name": n} for n in names])
            )

        return any_reagent

    rxn_df = pandas.DataFrame.from_dict(
        {
            "name": list(map(chemkin_io.writer.format_rxn_name, rxn_dct.keys())),
            "orig-name": list(map(original_reaction_name, rxn_dct.keys())),
            "reagent-names": [r + p for r, p, _ in rxn_dct.keys()],
        }
    )

    rcol = rxn_df["reagent-names"]
    rxn_df["any_stereo"] = rcol.progress_apply(any_reagent_("any_stereo"))
    rxn_df["atom_stereo"] = rcol.progress_apply(any_reagent_("atom_stereo"))
    rxn_df["bond_stereo"] = rcol.progress_apply(any_reagent_("bond_stereo"))
    rxn_df["non_inchi_stereo"] = rcol.progress_apply(any_reagent_("non_inchi_stereo"))
    rxn_df["res_bond_stereo"] = rcol.progress_apply(any_reagent_("res_bond_stereo"))
    rxn_df["vin_bond_stereo"] = rcol.progress_apply(any_reagent_("vin_bond_stereo"))
    return rxn_df

In [8]:
# 2d. function: calculate aggregate stats for each category
def aggregate_stats(df):
    has_fle = 'fleeting_atom_stereo' in df and 'fleeting_bond_stereo' in df
    fle = df['fleeting_atom_stereo'] | df['fleeting_bond_stereo'] if has_fle else None
    stats = pandas.Series({
        "Total": df.shape[0],
        "Stereo (any kind)": df['any_stereo'].sum(),
        "Atom stereo": df['atom_stereo'].sum(),
        "Bond stereo": df['bond_stereo'].sum(),
        "No InChI": df['non_inchi_stereo'].sum(),
        "Res. bond stereo": df['res_bond_stereo'].sum(),
        "Vin. bond stereo": df['vin_bond_stereo'].sum(),
        "Fleeting stereo": fle.sum() if has_fle else pandas.NA,
        "Fleeting atom stereo": df['fleeting_atom_stereo'].sum() if has_fle else pandas.NA,
        "Fleeting bond stereo": df['fleeting_bond_stereo'].sum() if has_fle else pandas.NA,
    }, dtype='UInt64')
    return stats

In [9]:
# 2e. function: calculate disaggregate stats for each category
def disaggregate_stats(df):
    any_stereo = df['any_stereo']
    atom_stereo = df['atom_stereo']
    bond_stereo = df['bond_stereo']
    res_bond_stereo = df['res_bond_stereo']
    vin_bond_stereo = df['vin_bond_stereo']
    non_inchi_stereo = df['non_inchi_stereo']

    has_fle = 'fleeting_atom_stereo' in df and 'fleeting_bond_stereo' in df
    fle_atom_stereo = df['fleeting_atom_stereo'] if has_fle else pandas.NA
    fle_bond_stereo = df['fleeting_bond_stereo'] if has_fle else pandas.NA

    inchi_stereo = any_stereo & ~non_inchi_stereo
    inchi_atom_stereo = atom_stereo & ~non_inchi_stereo & ~bond_stereo
    inchi_bond_stereo = bond_stereo & ~non_inchi_stereo & ~res_bond_stereo
    if has_fle:
        fle_atom_stereo &= ~fle_bond_stereo

    stats = pandas.Series({
        "InChI": inchi_stereo.sum(),
        "Atom stereo": inchi_atom_stereo.sum(),
        "Bond stereo": inchi_bond_stereo.sum(),
        "No InChI": non_inchi_stereo.sum(),
        "Res. bond stereo": res_bond_stereo.sum(),
        "Vin. bond stereo": vin_bond_stereo.sum(),
        "Fleeting atom stereo": fle_atom_stereo.sum() if has_fle else pandas.NA,
        "Fleeting bond stereo": fle_bond_stereo.sum() if has_fle else pandas.NA,
    }, dtype='UInt64')
    return stats

In [10]:
# 3a. (long-running) calculate species stats for the fully expanded DataFrame
FUEX_C67_SPC_DF = species_stats_columns(FUEX_C67_SPC_DF)
FUEX_PYR_SPC_DF = species_stats_columns(FUEX_PYR_SPC_DF)

  0%|          | 0/3907 [00:00<?, ?it/s]

  0%|          | 0/3907 [00:00<?, ?it/s]

  0%|          | 0/3907 [00:00<?, ?it/s]

  0%|          | 0/3907 [00:00<?, ?it/s]

  0%|          | 0/3907 [00:00<?, ?it/s]

...

  0%|          | 0/450 [00:00<?, ?it/s]

  0%|          | 0/450 [00:00<?, ?it/s]

  0%|          | 0/450 [00:00<?, ?it/s]

  0%|          | 0/450 [00:00<?, ?it/s]

  0%|          | 0/450 [00:00<?, ?it/s]

In [11]:
# 3b. add calculated stats to the original and NR DataFrames
FUEX_C67_SPC_DF["stereo-name"] = FUEX_C67_SPC_DF["name"]
FUEX_PYR_SPC_DF["stereo-name"] = FUEX_PYR_SPC_DF["name"]

ORIG_C67_SPC_DF = helpers.with_columns_from_other(
    ORIG_C67_SPC_DF, FUEX_C67_SPC_DF, ["orig-name"]
)
NREX_C67_SPC_DF = helpers.with_columns_from_other(
    NREX_C67_SPC_DF, FUEX_C67_SPC_DF, ["stereo-name"]
)
ORIG_PYR_SPC_DF = helpers.with_columns_from_other(
    ORIG_PYR_SPC_DF, FUEX_PYR_SPC_DF, ["orig-name"]
)
NREX_PYR_SPC_DF = helpers.with_columns_from_other(
    NREX_PYR_SPC_DF, FUEX_PYR_SPC_DF, ["stereo-name"]
)

In [12]:
# 3c. form tabls with aggregate / disaggregate stats for species
print('Species Aggregate Stats:')
SPC_AGG_STATS_DF = pandas.DataFrame.from_dict({
    'C6-7 Orig': aggregate_stats(ORIG_C67_SPC_DF),
    'C6-7 NR Exp': aggregate_stats(NREX_C67_SPC_DF),
    'C6-7 Full Exp': aggregate_stats(FUEX_C67_SPC_DF),
    'Pyro Orig': aggregate_stats(ORIG_PYR_SPC_DF),
    'Pyro NR Exp': aggregate_stats(NREX_PYR_SPC_DF),
    'Pyro Full Exp': aggregate_stats(FUEX_PYR_SPC_DF),
})
display(SPC_AGG_STATS_DF)

print('Species Disaggregate Stats:')
SPC_DISAGG_STATS_DF = pandas.DataFrame.from_dict({
    'C6-7 Orig': disaggregate_stats(ORIG_C67_SPC_DF),
    'C6-7 NR Exp': disaggregate_stats(NREX_C67_SPC_DF),
    'C6-7 Full Exp': disaggregate_stats(FUEX_C67_SPC_DF),
    'Pyro Orig': disaggregate_stats(ORIG_PYR_SPC_DF),
    'Pyro NR Exp': disaggregate_stats(NREX_PYR_SPC_DF),
    'Pyro Full Exp': disaggregate_stats(FUEX_PYR_SPC_DF),
})
display(SPC_DISAGG_STATS_DF)

Species Aggregate Stats:


,C6-7 Orig,C6-7 NR Exp,C6-7 Full Exp,Pyro Orig,Pyro NR Exp,Pyro Full Exp
Total,1676,2370,3907,309,420,450
Stereo (any kind),1187,1881,3416,102,213,240
Atom stereo,1046,1555,3090,24,28,55
Bond stereo,210,468,610,80,189,193
No InChI,73,189,211,44,116,116
Res. bond stereo,56,155,169,28,82,82
Vin. bond stereo,17,34,42,16,34,34
Fleeting stereo,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
Fleeting atom stereo,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
Fleeting bond stereo,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>


Species Disaggregate Stats:


,C6-7 Orig,C6-7 NR Exp,C6-7 Full Exp,Pyro Orig,Pyro NR Exp,Pyro Full Exp
InChI,1114,1692,3205,58,97,124
Atom stereo,977,1413,2806,22,24,47
Bond stereo,137,279,399,36,73,77
No InChI,73,189,211,44,116,116
Res. bond stereo,56,155,169,28,82,82
Vin. bond stereo,17,34,42,16,34,34
Fleeting atom stereo,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
Fleeting bond stereo,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>


In [13]:
# 4a. form reaction stats DataFrames based on species stats
ORIG_C67_RXN_DF = reaction_stats_columns(ORIG_C67_RXN_DCT, ORIG_C67_SPC_DF)
NREX_C67_RXN_DF = reaction_stats_columns(NREX_C67_RXN_DCT, NREX_C67_SPC_DF)
FUEX_C67_RXN_DF = reaction_stats_columns(FUEX_C67_RXN_DCT, FUEX_C67_SPC_DF)

ORIG_PYR_RXN_DF = reaction_stats_columns(ORIG_PYR_RXN_DCT, ORIG_PYR_SPC_DF)
NREX_PYR_RXN_DF = reaction_stats_columns(NREX_PYR_RXN_DCT, NREX_PYR_SPC_DF)
FUEX_PYR_RXN_DF = reaction_stats_columns(FUEX_PYR_RXN_DCT, FUEX_PYR_SPC_DF)

  0%|          | 0/3470 [00:00<?, ?it/s]

  0%|          | 0/3470 [00:00<?, ?it/s]

  0%|          | 0/3470 [00:00<?, ?it/s]

  0%|          | 0/3470 [00:00<?, ?it/s]

  0%|          | 0/3470 [00:00<?, ?it/s]

...

  0%|          | 0/1723 [00:00<?, ?it/s]

  0%|          | 0/1723 [00:00<?, ?it/s]

  0%|          | 0/1723 [00:00<?, ?it/s]

  0%|          | 0/1723 [00:00<?, ?it/s]

  0%|          | 0/1723 [00:00<?, ?it/s]

In [14]:
# 4b. (long-running) calculate fleeting stereo stats for the reactions
# (Requires info not stored in the mechanism file, to avoid recalculating)
C67_FLE_DF = fleeting_stereo_stats_columns(C67_RXN_DF)
PYR_FLE_DF = fleeting_stereo_stats_columns(PYR_RXN_DF)

  0%|          | 0/8891 [00:00<?, ?it/s]

  0%|          | 0/1723 [00:00<?, ?it/s]

In [15]:
# 4c. add racemic names to the fleeting stereo stats
NREX_C67_RXN_DF["racem-name"] = NREX_C67_RXN_DF["name"]
NREX_PYR_RXN_DF["racem-name"] = NREX_PYR_RXN_DF["name"]

C67_CHI_DCT = dict(zip(FUEX_C67_SPC_DF["name"], FUEX_C67_SPC_DF["inchi"]))
PYR_CHI_DCT = dict(zip(FUEX_PYR_SPC_DF["name"], FUEX_PYR_SPC_DF["inchi"]))

C67_FLE_DF["racem-name"] = C67_FLE_DF["name"].map(
    lambda n: helpers.racemic_reaction_name(n, C67_CHI_DCT)
)
PYR_FLE_DF["racem-name"] = PYR_FLE_DF["name"].map(
    lambda n: helpers.racemic_reaction_name(n, PYR_CHI_DCT)
)

In [16]:
# 4d. add fleeting stereo to reaction stats DataFrames
ORIG_C67_RXN_DF = helpers.with_columns_from_other(
    ORIG_C67_RXN_DF, C67_FLE_DF, ["orig-name"]
)
NREX_C67_RXN_DF = helpers.with_columns_from_other(
    NREX_C67_RXN_DF, C67_FLE_DF, ["racem-name"]
)
FUEX_C67_RXN_DF = helpers.with_columns_from_other(
    FUEX_C67_RXN_DF, C67_FLE_DF, ["name"]
)

ORIG_PYR_RXN_DF = helpers.with_columns_from_other(
    ORIG_PYR_RXN_DF, PYR_FLE_DF, ["orig-name"]
)
NREX_PYR_RXN_DF = helpers.with_columns_from_other(
    NREX_PYR_RXN_DF, PYR_FLE_DF, ["racem-name"]
)
FUEX_PYR_RXN_DF = helpers.with_columns_from_other(
    FUEX_PYR_RXN_DF, PYR_FLE_DF, ["name"]
)

In [17]:
# 4e. form tabls with aggregate / disaggregate stats for reactions
print('Reaction Aggregate Stats:')
RXN_AGG_STATS_DF = pandas.DataFrame.from_dict({
    'C6-7 Orig': aggregate_stats(ORIG_C67_RXN_DF),
    'C6-7 NR Exp': aggregate_stats(NREX_C67_RXN_DF),
    'C6-7 Full Exp': aggregate_stats(FUEX_C67_RXN_DF),
    'Pyro Orig': aggregate_stats(ORIG_PYR_RXN_DF),
    'Pyro NR Exp': aggregate_stats(NREX_PYR_RXN_DF),
    'Pyro Full Exp': aggregate_stats(FUEX_PYR_RXN_DF),
})
display(RXN_AGG_STATS_DF)

print('Reaction Disaggregate Stats:')
RXN_DISAGG_STATS_DF = pandas.DataFrame.from_dict({
    'C6-7 Orig': disaggregate_stats(ORIG_C67_RXN_DF),
    'C6-7 NR Exp': disaggregate_stats(NREX_C67_RXN_DF),
    'C6-7 Full Exp': disaggregate_stats(FUEX_C67_RXN_DF),
    'Pyro Orig': disaggregate_stats(ORIG_PYR_RXN_DF),
    'Pyro NR Exp': disaggregate_stats(NREX_PYR_RXN_DF),
    'Pyro Full Exp': disaggregate_stats(FUEX_PYR_RXN_DF),
})
display(RXN_DISAGG_STATS_DF)

Reaction Aggregate Stats:


,C6-7 Orig,C6-7 NR Exp,C6-7 Full Exp,Pyro Orig,Pyro NR Exp,Pyro Full Exp
Total,3470,5661,8891,1118,1589,1723
Stereo (any kind),2407,4598,7828,417,888,1022
Atom stereo,1894,3228,6458,106,137,271
Bond stereo,848,2269,3168,331,797,840
No InChI,473,1353,1608,263,659,686
Res. bond stereo,426,1260,1491,170,478,505
Vin. bond stereo,47,93,117,98,192,192
Fleeting stereo,773,1392,1704,327,559,578
Fleeting atom stereo,769,1386,1698,309,540,559
Fleeting bond stereo,4,6,6,18,19,19


Reaction Disaggregate Stats:


,C6-7 Orig,C6-7 NR Exp,C6-7 Full Exp,Pyro Orig,Pyro NR Exp,Pyro Full Exp
InChI,1934,3245,6220,154,229,336
Atom stereo,1559,2329,4660,86,91,182
Bond stereo,375,916,1560,68,138,154
No InChI,473,1353,1608,263,659,686
Res. bond stereo,426,1260,1491,170,478,505
Vin. bond stereo,47,93,117,98,192,192
Fleeting atom stereo,769,1386,1698,309,540,559
Fleeting bond stereo,4,6,6,18,19,19


In [18]:
# 5. write aggregate/disaggregate stats to CSV files
AGG_STATS_PATH = 'data/stats/07_agg.csv'
AGG_STATS_DF = pandas.concat([SPC_AGG_STATS_DF.add_suffix(' S'), RXN_AGG_STATS_DF.add_suffix(' R')], axis=1)
AGG_STATS_DF.to_csv(AGG_STATS_PATH)
display(AGG_STATS_DF)

DISAGG_STATS_PATH = 'data/stats/07_disagg.csv'
DISAGG_STATS_DF = pandas.concat([SPC_DISAGG_STATS_DF.add_suffix(' S'), RXN_DISAGG_STATS_DF.add_suffix(' R')], axis=1)
DISAGG_STATS_DF.to_csv(DISAGG_STATS_PATH)
display(DISAGG_STATS_DF)

,C6-7 Orig S,C6-7 NR Exp S,C6-7 Full Exp S,Pyro Orig S,Pyro NR Exp S,Pyro Full Exp S,C6-7 Orig R,C6-7 NR Exp R,C6-7 Full Exp R,Pyro Orig R,Pyro NR Exp R,Pyro Full Exp R
Total,1676,2370,3907,309,420,450,3470,5661,8891,1118,1589,1723
Stereo (any kind),1187,1881,3416,102,213,240,2407,4598,7828,417,888,1022
Atom stereo,1046,1555,3090,24,28,55,1894,3228,6458,106,137,271
Bond stereo,210,468,610,80,189,193,848,2269,3168,331,797,840
No InChI,73,189,211,44,116,116,473,1353,1608,263,659,686
Res. bond stereo,56,155,169,28,82,82,426,1260,1491,170,478,505
Vin. bond stereo,17,34,42,16,34,34,47,93,117,98,192,192
Fleeting stereo,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,773,1392,1704,327,559,578
Fleeting atom stereo,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,769,1386,1698,309,540,559
Fleeting bond stereo,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,4,6,6,18,19,19


,C6-7 Orig S,C6-7 NR Exp S,C6-7 Full Exp S,Pyro Orig S,Pyro NR Exp S,Pyro Full Exp S,C6-7 Orig R,C6-7 NR Exp R,C6-7 Full Exp R,Pyro Orig R,Pyro NR Exp R,Pyro Full Exp R
InChI,1114,1692,3205,58,97,124,1934,3245,6220,154,229,336
Atom stereo,977,1413,2806,22,24,47,1559,2329,4660,86,91,182
Bond stereo,137,279,399,36,73,77,375,916,1560,68,138,154
No InChI,73,189,211,44,116,116,473,1353,1608,263,659,686
Res. bond stereo,56,155,169,28,82,82,426,1260,1491,170,478,505
Vin. bond stereo,17,34,42,16,34,34,47,93,117,98,192,192
Fleeting atom stereo,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,769,1386,1698,309,540,559
Fleeting bond stereo,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,4,6,6,18,19,19
